In [1]:
import cv2
import numpy as np

##(1) read into  bgr-space
img = cv2.imread("3.jpg")

##(2) convert to hsv-space, then split the channels
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
h,s,v = cv2.split(hsv)

##(3) threshold the S channel using adaptive method(`THRESH_OTSU`) or fixed thresh
th, threshed = cv2.threshold(s, 50, 255, cv2.THRESH_BINARY_INV)

##(4) find all the external contours on the threshed S
#_, cnts, _ = cv2.findContours(threshed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = cv2.findContours(threshed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]

canvas  = img.copy()
#cv2.drawContours(canvas, cnts, -1, (0,255,0), 1)

## sort and choose the largest contour
cnts = sorted(cnts, key = cv2.contourArea)
cnt = cnts[-1]

## approx the contour, so the get the corner points
arclen = cv2.arcLength(cnt, True)
x,y,w,h=cv2.boundingRect(cnt)
print(x,y,w,h)
approx = cv2.approxPolyDP(cnt, 0.02* arclen, True)
# cv2.drawContours(canvas, [cnt], -1, (255,0,0), 3, 1)
cv2.drawContours(canvas, [approx], -1, (0, 0, 255), 3)


## Ok, you can see the result as tag(6)
cv2.imwrite("detected5.png", canvas)

78 634 2946 3114


True

In [26]:
!pip install imutils

In [2]:
import cv2 # opencv-python
import numpy as np
from skimage.filters import threshold_local # scikit-image
import imutils

In [4]:
# read the input image
image_org = cv2.imread("3.jpg")
print(image_org.shape)
# clone the original image
original_image = image_org.copy()

# resize using ratio (old height to the new height)
ratio = image_org.shape[0] / 500.0
image = imutils.resize(image_org, height=500)

(4032, 3024, 3)


In [5]:
image.shape[0]

500

In [6]:
#  change the color space to YUV
image_yuv = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)

# grap only the Y component
image_y = np.zeros(image_yuv.shape[0:2], np.uint8)
image_y[:, :] = image_yuv[:, :, 0]

In [7]:
print(image_yuv.shape)

(500, 375, 3)


In [12]:
cv2.imwrite("img_y.png",image_y)

True

In [8]:
# blur the image to reduce high frequency noises
image_blurred = cv2.GaussianBlur(image_y, (3, 3), 0)

In [9]:
## 이거도 잘된다
image_blurred = cv2.GaussianBlur(image_yuv, (3, 3), 0)

In [10]:
# find edges in the image
edges = cv2.Canny(image_blurred, 50, 200, apertureSize=3)
cv2.imwrite("edge.png", edges)

True

In [11]:
# find contours
contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# print(contours)
# draw all contours on the original image
#cv2.drawContours(image, contours, -1, (0, 255, 0), 1)

In [12]:
# to collect all the detected polygons
polygons = []

# loop over the contours
for cnt in contours:
    # find the convex hull
    hull = cv2.convexHull(cnt)
    
    # compute the approx polygon and put it into polygons
    #polygons.append(cv2.approxPolyDP(hull, 0.01 * cv2.arcLength(hull, True), False))
    polygons.append(cv2.approxPolyDP(cnt, 0.01 * cv2.arcLength(cnt, True), False))
    print(polygons)

[array([[[374, 235]],

       [[225, 497]],

       [[304, 372]],

       [[374, 236]]], dtype=int32)]
[array([[[374, 235]],

       [[225, 497]],

       [[304, 372]],

       [[374, 236]]], dtype=int32), array([[[ 52, 170]],

       [[ 10, 235]],

       [[ 12, 239]],

       [[ 51, 171]]], dtype=int32)]
[array([[[374, 235]],

       [[225, 497]],

       [[304, 372]],

       [[374, 236]]], dtype=int32), array([[[ 52, 170]],

       [[ 10, 235]],

       [[ 12, 239]],

       [[ 51, 171]]], dtype=int32), array([[[  0, 161]],

       [[ 27, 178]],

       [[  0, 225]],

       [[ 27, 181]],

       [[ 26, 176]],

       [[  1, 161]]], dtype=int32)]
[array([[[374, 235]],

       [[225, 497]],

       [[304, 372]],

       [[374, 236]]], dtype=int32), array([[[ 52, 170]],

       [[ 10, 235]],

       [[ 12, 239]],

       [[ 51, 171]]], dtype=int32), array([[[  0, 161]],

       [[ 27, 178]],

       [[  0, 225]],

       [[ 27, 181]],

       [[ 26, 176]],

       [[  1, 161]]], dtyp

In [22]:
print(cv2.contourArea(sortedPoly[2]))

421.5


In [26]:
# sort polygons in desc order of contour area
sortedPoly = sorted(polygons, key=cv2.contourArea, reverse=True)
print(sortedPoly[0])

rect=cv2.minAreaRect(sortedPoly[0])
box=cv2.boxPoints(rect)
print(box)
box=np.int0(box)
print(box)

cv2.drawContours(image, [box],0, (0,255,0),2)
cv2.circle(image, (box[0][0],box[0][1]), 1, (0,0,255), 10)
cv2.circle(image, (box[1][0],box[1][1]), 1, (0,0,255), 10)
cv2.circle(image, (box[2][0],box[2][1]), 1, (0,0,255), 10)
cv2.circle(image, (box[3][0],box[3][1]), 1, (0,0,255), 10)
#cv2.rectangle(image,(x,y),(x+w,y+h),(0,255,0),2)
# draw points of the intersection of only the largest polyogon with red color
#cv2.drawContours(image, sortedPoly[0], -1, (0, 0, 255), 5)
#cv2.drawContours(image, box, -1, (0, 0, 255), 5)

cv2.imwrite("detected3.png", image)

[[[191  78]]

 [[125 115]]

 [[ 10 365]]

 [[228 463]]

 [[366 156]]

 [[192  79]]]
[[228.       463.00006 ]
 [  9.997551 365.0055  ]
 [147.99756   58.005432]
 [366.       156.      ]]
[[228 463]
 [  9 365]
 [147  58]
 [366 156]]


True

In [85]:
rect=cv2.boundingRect(sortedPoly[0])
rect

(23, 70, 338, 388)

In [54]:
sortedPoly[0][1][0][0], sortedPoly[0][1][0][1]

(29, 446)

In [49]:
sortedPoly[0][0]

array([[340, 445]], dtype=int32)

In [50]:
len(sortedPoly[0])

5

In [65]:
box.shape

AttributeError: 'list' object has no attribute 'shape'

In [55]:
sortedPoly[0].shape

(5, 1, 2)

In [56]:
sortedPoly[0]

array([[[340, 445]],

       [[ 29, 446]],

       [[ 51,  74]],

       [[333,  88]],

       [[340, 381]]], dtype=int32)

In [77]:
sortedPoly

[array([[[360, 153]],
 
        [[239, 457]],
 
        [[ 23, 370]],
 
        [[142,  70]],
 
        [[360, 152]]], dtype=int32), array([[[140,  40]],
 
        [[  0, 280]],
 
        [[ 97,   0]],
 
        [[140,  33]]], dtype=int32), array([[[ 41, 188]],
 
        [[  0, 257]],
 
        [[  0, 158]],
 
        [[ 41, 182]]], dtype=int32), array([[[ 39, 187]],
 
        [[  0, 253]],
 
        [[  0, 161]],
 
        [[ 39, 184]]], dtype=int32), array([[[374, 239]],
 
        [[309, 370]],
 
        [[231, 499]],
 
        [[374, 238]]], dtype=int32), array([[[20, 23]],
 
        [[ 7, 46]],
 
        [[ 4, 49]],
 
        [[ 0, 49]],
 
        [[ 0, 15]],
 
        [[ 5, 11]],
 
        [[20, 19]]], dtype=int32), array([[[58, 24]],
 
        [[52, 34]],
 
        [[49, 37]],
 
        [[43, 37]],
 
        [[32, 30]],
 
        [[32, 27]],
 
        [[39, 16]],
 
        [[43, 14]],
 
        [[58, 22]]], dtype=int32), array([[[69,  5]],
 
        [[60, 20]],
 
        [[43, 11